In [16]:
import pandas as pd
import nltk
import re
import numpy as np
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 
from nltk.corpus import stopwords
import pickle
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yolandaferreirofranchi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
#import pre-processed data from pickle 
df_09= pd.read_pickle(r"/Users/yolandaferreirofranchi/Documents/GitHub/Masters-Thesis/sample_results_df_09.pickle")
df_09

,pre_processed_sent,male_count,female_count,apicall_fail,sentences,article_id,year,col_type
0,"[break, silence, surround, break, say, band, l...",3,0,0,Liam Gallagher has broken the silence surround...,5048,2009,0
1,"[however, interview, say, longer]",2,0,0,"However, in an interview with The Times Liam G...",5048,2009,0
9,"[leave, band, follow, bust, say, simply, could...",4,0,0,Noel Gallagher left the Manchester band follow...,5048,2009,0
10,"[launch, clothing, line, earlier, year, admit,...",3,0,0,"""Liam launched his clothing line Pretty Green ...",5048,2009,0
13,"[people, able, buy, record]",1,0,0,"""People will be able to buy his records.",5048,2009,0
...,...,...,...,...,...,...,...,...
9706,"[organisers, say, extend, programme, live, tou...",2,0,0,Organisers of the X Factor have said they've e...,1043733,2009,0
9707,"[vote, judge, week, seven, competition]",1,0,0,Jedward were voted off by the X Factor judges ...,1043733,2009,0
9708,"[two, month, tour, begin, see, extra, date, add]",3,0,0,"The two month tour, which begins in Liverpool ...",1043733,2009,0
9709,"[artists, confirm, tour, contestant]",2,1,0,Artists confirmed for the tour are contestants...,1043733,2009,0


In [ ]:
#NO NEED FOR THIS ANYMORE GIVEN PP
def split_df(df, column_name): 
    """Split Sentences DF by the year column so that we can do the LR by year"""

    #get all unique values in col
    col_vals = df[column_name].unique()

    split = [df[df[column_name] == value] for value in col_vals]
    
    #Return the list of split dataframes
    return split

#use the new function to split the original sentences df 
splitted_df = split_df(sentences_df, 'year')

#print each of the DFs
for df_year_data in splitted_df:
    display(df_year_data)

**Binary Logistic Regression**

In [11]:
#vectorizer = TfidfVectorizer(max_features= 1000, lowercase=False, tokenizer=False)
def fake(token):
    return token

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=fake,
    preprocessor=fake,
    token_pattern=None)  

#X_train = tfidf.fit_transform(X_train)
#X_test = tfidf.fit_transform(X_test)

In [ ]:
#GridSearchCV Packages - this still needs changing and fitting into the LR function!
from sklearn.model_selection import GridSearchCV
import warnings 
warnings.filterwarnings("ignore")


# parameter grid
parameters = {"penalty": ['l1','l2'], 
              "C": np.logspace(-3,3,7),
              "solver": ['newton-cg', 'lbfgs', 'liblinear'],
}

#GridSearchCV
logreg = LogisticRegression()
clf_logreg = GridSearchCV(logreg, 
                          param_grid = parameters, 
                          scoring = "accuracy", 
                          cv = 10)

clf_logreg.fit(X_train, y_train)

**Logistic Regression Classifier by Year**

In [27]:
def logistic_regression_year(df, target_col, text_col):
    #split data 
    X = df[text_col].apply(lambda x: str(x))
    y = df[target_col]

    #train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    tfidf = TfidfVectorizer()
    X_train_tfidf = tfidf.fit_transform(X_train)
    X_test_tfidf = tfidf.transform(X_test)

    #run the classifier 
    clf = LogisticRegression()
    clf.fit(X_train_tfidf, y_train)
    y_pred = clf.predict(X_test_tfidf)

    #performance 
    accuracy = clf.score(X_test_tfidf, y_test)
    class_report = classification_report(y_test, y_pred, zero_division = 0)
    results = {'accuracy': accuracy, 'classification_report': class_report}
    #print(f"Year: {year}")
    print(f"Accuracy: {accuracy}")
    print(f"Classification Report:\n{class_report}")

        #coefficients
    coefs = clf.coef_[0]
    sorted_coef = sorted((zip(tfidf.get_feature_names_out(), coefs)), key = lambda x: x[1], reverse=True)
    high_coef = sorted_coef[:10]
    low_coef = sorted_coef[-10:]
        

        #print coefficient results 
    print(f"\n Highest coefs:")
    for i in high_coef: 
        print(i)
        
    print(f"\n Lowest coefs:")
    for i in low_coef: 
        print(i)
    

In [28]:
logistic_regression_year(df_09, 'col_type', 'pre_processed_sent')

Accuracy: 0.6801801801801802
Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.91      0.78       418
           1       0.66      0.29      0.40       248

    accuracy                           0.68       666
   macro avg       0.67      0.60      0.59       666
weighted avg       0.68      0.68      0.64       666


 Highest coefs:
('say', 2.7058098234812835)
('newsbeat', 1.625300425913965)
('really', 1.5431860499940846)
('time', 1.4287925439957982)
('add', 1.3693210892807677)
('cool', 1.3223848060170107)
('end', 1.2985914543875967)
('winner', 1.2865737872946597)
('pregnant', 1.2502019510388829)
('feel', 1.225859452412414)

 Lowest coefs:
('festival', -1.0237310431052795)
('game', -1.074189310079536)
('travel', -1.0757708610895875)
('british', -1.09962095953621)
('film', -1.1014707485611273)
('forthcoming', -1.147294793064275)
('country', -1.1637579011533543)
('hold', -1.2145405069367237)
('include', -1.3823713793188768)
('b